In [1]:
# hide
%load_ext autoreload
%autoreload 2

In [1]:
import random

import numpy as np
import pandas as pd
import torch
from clientClass import Client

from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.tree import DecisionTreeClassifier
# del sys.modules['cryptotree.tree']
# del sys.modules['cryptotree.preprocessing']
# del sys.modules['cryptotree.polynomials']
# del sys.modules['cryptotree.cryptotree']
import builtins

import tenseal.sealapi as seal
from cryptotree.cryptotree import (HomomorphicNeuralRandomForest,
                                   HomomorphicTreeEvaluator,
                                   HomomorphicTreeFeaturizer)
from cryptotree.polynomials import (plot_graph_function_approximation,
                                    polyeval_tree)
# del sys.modules['cryptotree.seal_helper']
from cryptotree.preprocessing import Featurizer
from cryptotree.seal_helper import (append_globals_to_builtins,
                                    create_seal_globals, print_ctx,
                                    print_range_ctx, print_vector)
from cryptotree.tree import (CrossEntropyLabelSmoothing, NeuralRandomForest,
                             SigmoidTreeMaker, TanhTreeMaker)
from dataFunction import data_prep, import_data, make_dummies, new_df
from fastai.basic_data import DataBunch
from fastai.metrics import accuracy
from fastai.tabular.learner import Learner
from HE_functions import *
from sklearn.ensemble import RandomForestClassifier
from torch.utils import data
import missingno as msno
# nest_asyncio.apply()
# np.random.seed(10)
# tf.random.set_seed(10)
# tff.framework.set_default_context(tff.backends.native.create_thread_debugging_execution_context(clients_per_thread=50))


# Make Dataframes for every dataset

In [2]:
cleveland = "processed.cleveland.data"
switzerland = "processed.switzerland.data"
va = "processed.va.data"
hungarian = "reprocessed.hungarian.data"
cleveland_df, switzerland_df, va_df, hungarian_df = import_data(cleveland, switzerland, va, hungarian)
# df = new_df(cleveland_df, switzerland_df, va_df, hungarian_df)
df_dict ={
    'Cleveland': cleveland_df,
    'Switzerland': switzerland_df,
    'VA Long Beach': va_df,
    'Hungary': hungarian_df        
    }



In [4]:
for i in clients:
    sample = np.random.randint(1/2*len(i.y), len(i.y))
    print(sample)
    print(chance_of_dead_tree(sample, sum(i.y == 0), sum(i.y == 1)))

NameError: name 'clients' is not defined

In [3]:
torch.random.manual_seed(73)
random.seed(73)
n_clients = 4
# df_dict ={
#     'Cleveland': cleveland_df,
#     'Switzerland': switzerland_df,
#     'VA Long Beach': va_df,
#     'Hungary': hungarian_df        
#     }
clients = []
for i in list(df_dict.keys()):
    df = df_dict.get(i)
    df = new_df(df)
    location_data = df
    # location_data = new_df(df_dict.get(i))
    y = location_data.HeartDisease
    # y =  torch.tensor(location_data["HeartDisease"].values).float().unsqueeze(1)
    location_data = location_data.drop(columns="HeartDisease")
    cat_feat = ['ChestPainType', 'RestingECG', 'ST_Slope']
    location_data = make_dummies(location_data, cat_feat)
    numeric_feature_names = ['Age', 'MaxHR', 'RestingBP',  'Cholesterol', 'Oldpeak']
    for j in numeric_feature_names:
        if location_data[j].std() != 0: 
            location_data[j] = (location_data[j] - location_data[j].mean()) / location_data[j].std()
    x = location_data
    # x = torch.tensor(location_data.values).float()
    clients.append(Client(i, x, y, cat_feat))


# X_train, X_test, y_train, y_test = train_test_split(clients[0].X, clients[0].y, test_size=0.2, random_state=42)

In [ ]:
# del sys.modules['HE_functions']

In [ ]:
clients[1].X

In [ ]:
# fig, ax = plt.subplots(figsize = (12,8))
# artists = plot_tree(switzerland_estimators[1], ax=ax)
# for clf in switzerland_estimators:
#     # if clf.tree_.node_count < 5:    
#     fig, ax = plt.subplots(figsize = (12,8))
#     artists = plot_tree(clf, ax=ax)

In [ ]:
from HE_functions import *
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
dict_outputs = {}
feature_imp = {}
for i in clients:
    pipe = Featurizer(cat_feat)
    rf, sigmoid_tree_maker, tanh_tree_maker = set_tree(5, 13)
    y_train, y_valid, X_train_normalized, X_valid_normalized = split_prep_data(i, pipe)
    train_ds, valid_ds = make_tabularDataset(X_train_normalized, y_train, X_valid_normalized, y_valid)
    train_dl, valid_dl, fix_dl = make_dataloader(train_ds, valid_ds)
    rf.fit(X_train_normalized, y_train)
    estimators = rf.estimators_
    print('\n')
    print(''+i.name+ ': results')
    print('\n')

    rf, sigmoid_neural_rf, tanh_neural_rf = results_trees(rf, sigmoid_tree_maker, tanh_tree_maker, X_train_normalized, y_train)

    model = fine_tune_tree(tanh_tree_maker, rf, train_dl, valid_dl, fix_dl)

    pred = rf.predict(X_train_normalized)

    with torch.no_grad():
        neural_pred = model(torch.tensor(X_train_normalized).float()).argmax(dim=1).numpy()

    print(f"Original accuracy : {(pred == y_train).mean()}")
    print(f"Accuracy : {(neural_pred == y_train).mean()}")
    print(f"Same output : {(neural_pred == pred).mean()}")
    set_config()
    tree_evaluator, homomorphic_featurizer = setup_HE(tanh_tree_maker, model)

    hrf_pred = HE_RF_pred(X_valid_normalized, homomorphic_featurizer, tree_evaluator)
    
    nrf_pred = NRF_pred(valid_dl, model)

    linear_pred = LIN_pred(X_train_normalized, y_train, X_valid_normalized)

    # We compute the random forest predictions
    rf_pred = rf.predict(X_valid_normalized)
    feature_imp[i.name] = pd.Series(rf.feature_importances_,index=i.X.columns).sort_values(ascending=False)

    models = dict(nrf=nrf_pred,hrf=hrf_pred, rf=rf_pred, linear=linear_pred)

    outputs = []
    for name, pred in models.items():
        metrics = compute_metrics(pred, y_valid)
        metrics["model"] = name
        outputs.append(metrics)
    
    outputs = pd.DataFrame(outputs)
    dict_outputs[i.name] = outputs
    # outputs
    #     for p in model.parameters():
    #         print(p.shape, p.requires_grad)

    


# Random forest problems

sample split gives same y values when switzerland dataset is used. Since the heartdisease variable is unevenly divided. 

P(altijd 1) bij 50 subsamples == 2% als verdeling 114 keer 1 en 8 keer 0



P(altijd 1) bij 57 subsamples == 1%

In [ ]:
for i in dict_outputs.keys():
    print(i)
    print(dict_outputs[i])

In [ ]:
y_train, y_valid, X_train_normalized, X_valid_normalized = split_prep_data(clients[1], pipe)

In [ ]:
x = X_train_normalized[0]
ctx = homomorphic_featurizer.encrypt(x)
ptx = seal.Plaintext()
decryptor.decrypt(ctx, ptx)
print(ctx.save('./text'))

In [ ]:
%matplotlib inline
from sklearn.tree import plot_tree
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
estimator = DecisionTreeClassifier(max_depth=4, random_state=0)
estimator.fit(X_train_normalized, y_train)
fig, ax = plt.subplots(figsize = (12,8))
artists = plot_tree(estimator, ax=ax)

In [ ]:
# rf, sigmoid_neural_rf, tanh_neural_rf
feature_imp = pd.Series(rf.feature_importances_,index=clients[0].X.columns).sort_values(ascending=False)
feature_imp

In [ ]:
%matplotlib inline
plt.style.use('seaborn-darkgrid')
palette = plt.get_cmap('tab20')
plt.rcParams.update({'font.size': 16,})
num=1
f2= plt.figure(figsize=(50,30))
for i in list(df_dict.keys()):
    df = df_dict.get(i)
    for var in list(df.columns.array):
        ax = f2.add_subplot(4,5,num)
        a = np.asarray(df[var].dropna())
        ax.hist(a, bins=50, color=palette(num))
        ax.set_title(var)
        num+=1

In [11]:
# plt.rcParams["figure.figsize"] = (10,8)
# plt.style.use('seaborn-darkgrid')
# palette = plt.get_cmap('tab20')
# plt.rcParams.update({'font.size': 16,})
# num=1
# plt.show()
# f2= plt.figure(figsize=(50,30))
for i in list(df_dict.keys()):
    df = df_dict.get(i)
#     # for var in list(df.columns.array):
#     ax = f2.add_subplot(4,5,num)
#     a = np.asarray(df.HeartDisease.dropna())
#     ax.hist(a, bins=50, color=palette(num))
#     ax.set_title("" + i+ "heart disease")
#     num+=1
    print(i)
    print(df.HeartDisease.value_counts())


Cleveland
0    164
1    139
Name: HeartDisease, dtype: int64
Switzerland
1    115
0      8
Name: HeartDisease, dtype: int64
VA Long Beach
1    149
0     51
Name: HeartDisease, dtype: int64
Hungary
0.0    188
1.0    106
Name: HeartDisease, dtype: int64


In [ ]:
from testEncryp import *

# Scheme's parameters
# polynomial modulus degree
n = 2**4
# ciphertext modulus
q = 2**15
# plaintext modulus
t = 2**8
# polynomial modulus
poly_mod = np.array([1] + [0] * (n - 1) + [1])
# Keygen
pk, sk = keygen(n, q, poly_mod)
# Encryption
pt1, pt2 = 73, 20
cst1, cst2 = 7, 5
ct1 = encrypt(pk, n, q, t, poly_mod, pt1)
ct2 = encrypt(pk, n, q, t, poly_mod, pt2)

print("[+] Ciphertext ct1({}):".format(pt1))
print("")
print("\t ct1_0:", ct1[0])
print("\t ct1_1:", ct1[1])
print("")
print("[+] Ciphertext ct2({}):".format(pt2))
print("")
print("\t ct1_0:", ct2[0])
print("\t ct1_1:", ct2[1])
print("")

# Evaluation
ct3 = add_plain(ct1, cst1, q, t, poly_mod)
ct4 = mul_plain(ct2, cst2, q, t, poly_mod)

# Decryption
decrypted_ct3 = decrypt(sk, n, q, t, poly_mod, ct3)
decrypted_ct4 = decrypt(sk, n, q, t, poly_mod, ct4)

print("[+] Decrypted ct3(ct1 + {}): {}".format(cst1, decrypted_ct3))
print("[+] Decrypted ct4(ct2 * {}): {}".format(cst2, decrypted_ct4))